In [6]:
import pickle
import tensorflow
from keras import Model
from keras.layers import (
    Input,
    TimeDistributed,
    LSTM,
    Dense,
    Input,
    Dense,
    Conv1D,
    Flatten, 
    Embedding,
    Dropout)
from imblearn.over_sampling import SMOTE
import numpy
from numba import cuda 
device = cuda.get_current_device()
device.reset()
import tensorflow as tf

In [2]:
%run constants.py


In [3]:
with open(X_TRAIN_INPUT_SAVE_FILE_PRE_VEC, "rb") as f:
        x_train = pickle.load(f)
with open(Y_TRAIN_INPUT_SAVE_FILE, "rb") as f:
        y_train = pickle.load(f)
with open(X_VAL_INPUT_SAVE_FILE_PRE_VEC, "rb") as f:
        x_val = pickle.load(f)
with open(Y_VAL_INPUT_SAVE_FILE, "rb") as f:
        y_val = pickle.load(f)
with open(X_TEST_INPUT_SAVE_FILE_PRE_VEC, "rb") as f:
        x_test = pickle.load(f)
with open(Y_TEST_INPUT_SAVE_FILE, "rb") as f:
        y_test = pickle.load(f)

with open(EMBEDDING_MATRIX_SAVE_FILE, "rb") as f:
        embedding_matrix = pickle.load(f)

print("------Saving varaibles for reuse ------")
print(f"X_train shape: {x_train.shape}")
print(f"Y_train shape: {y_train.shape}")
print(f"X_test shape: {x_test.shape}")
print(f"Y_test shape: {y_test.shape}")
print(f"X_val shape: {x_val.shape}")
print(f"Y_val shape: {y_val.shape}")
print(f"Embedding shape: {embedding_matrix.shape}")
print(f"Total 0 values: {(y_train == 0).sum()}")
print(f"Total 1 values: {(y_train == 1).sum()}")

------Saving varaibles for reuse ------
X_train shape: (2465, 30)
Y_train shape: (2465,)
X_test shape: (685, 30)
Y_test shape: (685,)
X_val shape: (274, 30)
Y_val shape: (274,)
Embedding shape: (10000, 100)
Total 0 values: 1346
Total 1 values: 1119


In [4]:
# if BALANCE_DATA:
#     arr = x_train.reshape(len(x_train), -1)
#     print(arr.shape)
#     sm = SMOTE(random_state=SEED)
#     x_train_bal, y_train = sm.fit_resample(arr, y_train.ravel())
#     print(f"After OverSampling, the shape of train_X: {x_train_bal.shape}")
#     print(f"After OverSampling, the shape of train_y: {y_train.shape}")
#     print(f"After OverSampling, counts of label '1': {sum(y_train == 1)}")
#     print(f"After OverSampling, counts of label '0': {sum(y_train == 0)}")
#     x_train = numpy.reshape(x_train_bal, (-1, x_train.shape[1], x_train.shape[2]))
# x_train.shape


AttributeError: 'DataFrame' object has no attribute 'reshape'

In [8]:
train_corpora = x_train[29].str.split()
train_corpora = train_corpora.tolist()
flat_list_train_corpora = [x for xs in train_corpora for x in xs]
flat_list_train_corpora = list(set(flat_list_train_corpora))
from keras.layers import TextVectorization

def create_textvectorisation(lst):
    text_vectorization: TextVectorization = TextVectorization(
        output_mode="int",
        split="whitespace",
        max_tokens=MAX_VOCAB_SIZE,
        output_sequence_length=MAX_SEQUENCE_LENGTH,
    )
    text_vectorization.adapt(lst)
    return text_vectorization


text_vectorization = create_textvectorisation(flat_list_train_corpora)

In [9]:
import tqdm
def embed_vectors(text_vectorization):
    embeddings_index = {}

    f = open(GLOVE_100D_FILEPATH)
    for line in tqdm.tqdm(f, ncols=100, desc="Loading Glove Embeddings."):
        values = line.split()
        word = values[0]
        coefs = numpy.asarray(values[1:], dtype="float32")
        embeddings_index[word] = coefs
    f.close()

    print(f"Found {len(embeddings_index)} word vectors.")

    vocabulary = text_vectorization.get_vocabulary()
    word_index = dict(zip(vocabulary, range(len(vocabulary))))
    embedding_matrix = numpy.zeros((MAX_VOCAB_SIZE, EMBEDDING_DIM))

    for word, i in tqdm.tqdm(word_index.items(), desc="Embedding Matrix."):
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    return embedding_matrix

embedding_matrix = embed_vectors(text_vectorization)

Loading Glove Embeddings.: 400000it [00:05, 66836.66it/s]


Found 400000 word vectors.


Embedding Matrix.: 100%|██████████| 10000/10000 [00:00<00:00, 651329.90it/s]


In [10]:
embedding_layer = Embedding(input_dim=MAX_VOCAB_SIZE, output_dim=EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH, weights = [embedding_matrix], trainable=False)

In [27]:
X_train_indices = text_vectorization(x_train[29].to_list())
X_test_indices = text_vectorization(x_test[29].to_list())
X_val_indices = text_vectorization(x_val[29].to_list())

# train_dataset = tf.data.Dataset.from_tensor_slices((X_train_indices, y_train))
# train_dataset = train_dataset.batch(batch_size=BATCH_SIZE)
# test_dataset = tf.data.Dataset.from_tensor_slices((X_train_indices, y_train))
# test_dataset = test_dataset.batch(batch_size=BATCH_SIZE)
# val_dataset = tf.data.Dataset.from_tensor_slices((X_val_indices, y_val))
# val_dataset = val_dataset.batch(batch_size=BATCH_SIZE)


for column in x_train:
    print(text_vectorization(x_train[column].to_list()))




tf.Tensor(
[[6178    0    0 ...    0    0    0]
 [6178    0    0 ...    0    0    0]
 [6178    0    0 ...    0    0    0]
 ...
 [6178    0    0 ...    0    0    0]
 [6178    0    0 ...    0    0    0]
 [6178    0    0 ...    0    0    0]], shape=(2465, 200), dtype=int64)


TypeError: Exception encountered when calling layer "text_vectorization" (type TextVectorization).

Cannot convert a list containing a tensor of dtype <dtype: 'float64'> to <dtype: 'float32'> (Tensor is: <tf.Tensor: shape=(), dtype=float64, numpy=nan>)

Call arguments received by layer "text_vectorization" (type TextVectorization):
  • inputs=['tf.Tensor(shape=(), dtype=float32)', 'tf.Tensor(shape=(), dtype=float32)', 'tf.Tensor(shape=(), dtype=float32)', "'Consultation Post- Op wound healed well - no swelling/discharge, unable to palpate as dog very distressed, mild infmmation/redness as expetcted adv revisit one week. angel can then go through results and last popc'", 'tf.Tensor(shape=(), dtype=float32)', 'tf.Tensor(shape=(), dtype=float32)', 'tf.Tensor(shape=(), dtype=float32)', "'pre operative check Presented for pre-op check\\r\\n\\r\\nOR no seen in season, EDDU well , no concerns.\\r\\n\\r\\nPE: all unremarkable.\\r\\n\\r\\nAll fine, ok to book in for spay'", 'tf.Tensor(shape=(),

In [18]:
def run_stacked_lstm():
    document_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype="int32")
    embedding_sequences =embedding_layer(document_input)
    x = LSTM(32, return_sequences=True)(embedding_sequences)
    x = LSTM(32)(x)
    x = Dropout(0.3)(x)
    doc_model = Model(document_input, x)
    input_docs = Input(
                shape=(TIME_STEP, MAX_SEQUENCE_LENGTH), name="input_docs", dtype="int32"
            )

    x = TimeDistributed(doc_model, name="token_embedding_model")(input_docs)
    x = LSTM(32)(x)
    x = Dropout(0.3)(x)
    outputs = Dense(1, activation="sigmoid")(x)
    model = Model(input_docs, outputs)

    opt = tensorflow.keras.optimizers.Adam(learning_rate=LR, beta_1=0.5, beta_2=0.999)

    model.compile(
                loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"]
            )
    model.summary()
    model.fit(x=train_ds ,batch_size=BATCH_SIZE,
            epochs=NUM_EPOCHS, validation_data=val_ds)

In [7]:
def run_lstm():
    document_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype="int32")
    embedding_sequences =embedding_layer(document_input)
    x = LSTM(32)(embedding_sequences)
    x = Dropout(0.3)(x)
    doc_model = Model(document_input, x)
    input_docs = Input(
                shape=(TIME_STEP, MAX_SEQUENCE_LENGTH), name="input_docs", dtype="int32"
            )

    x = TimeDistributed(doc_model, name="token_embedding_model")(input_docs)
    x = LSTM(32)(x)
    x = Dropout(0.3)(x)
    outputs = Dense(1, activation="sigmoid")(x)
    model = Model(input_docs, outputs)

    opt = tensorflow.keras.optimizers.Adam(learning_rate=LR, beta_1=0.5, beta_2=0.999)

    model.compile(
                loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"]
            )
    model.summary()
    model.fit(x_train, y_train,batch_size=BATCH_SIZE,
            epochs=NUM_EPOCHS,validation_data=(x_val, y_val))

In [8]:
def run_cnn():
    document_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype="int32")
    embedding_sequences = embedding_layer(document_input)

    x = Conv1D(filters=300, kernel_size=5, padding="valid")(embedding_sequences)
    x = Dropout(0.3)(x)
    doc_model = Model(document_input, x)

    input_docs = Input(
            shape=(TIME_STEP, MAX_SEQUENCE_LENGTH), name="input_docs", dtype="int32"
        )

    x = TimeDistributed(doc_model, name="token_embedding_model")(input_docs)
    x = Conv1D(filters=300, kernel_size=5, padding="valid")(x)
    x = Dropout(0.3)(x)
    x = Flatten()(x)
    outputs = Dense(1, activation="sigmoid")(x)

    model = Model(input_docs, outputs)

    opt = tensorflow.keras.optimizers.Adam(learning_rate=LR, beta_1=0.5, beta_2=0.999)
    model.compile(
                loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"]
            )
    model.summary()
    model.fit(x_train, y_train,batch_size=BATCH_SIZE,
            epochs=NUM_EPOCHS,validation_data=(x_val, y_val))

In [19]:

run_stacked_lstm()
# run_lstm()
# run_cnn()

ValueError: Input 0 of layer "lstm_8" is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: (None, 100)